# **Flights Database Preprocess S12-S13-S15** 
This Jupyter Notebook is common for all the projects related with the diferent machine learning aproaches. 
In this notebook Flights_DB is preprocessed in order to Determine the correct Dtypes of each attributes and Deal with Null values.

In [1]:
# Imoporting packages
# ==============================================================================
import pandas as pd 
import datetime as dt
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
import math

from sklearn import preprocessing as prp
from sklearn import model_selection as ms
from sklearn import linear_model as lin_mod
from sklearn import ensemble
from sklearn import metrics
#import sklearn.pipeline as pipeline 

from lazypredict.Supervised import LazyRegressor

# Pandas Configuration
# ==============================================================================
pd.set_option('float_format', '{:.2f}'.format) # float numbers with 2 decimals
pd.set_option("display.max_columns", None) # Display all columns

# Seaborn configuration
# ==============================================================================
sns.set_theme(style='darkgrid', palette='deep')
dims = (20, 16)

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Import the CSV file as pandas Dataframe:

flights_raw_db = pd.read_csv("../db/Flights/DelayedFlights.csv", index_col=0)

## Dataset Features

**Year** 2008  
**Month** 1-12  
**DayofMonth** 1-31  
**DayOfWeek** 1 (Monday) - 7 (Sunday)  
**DepTime** actual departure time (local, hhmm)  
**CRSDepTime** scheduled departure time (local, hhmm)  
**ArrTime** actual arrival time (local, hhmm)  
**CRSArrTime** scheduled arrival time (local, hhmm)   
**UniqueCarrier** unique carrier code FlightNum flight number  
**TailNum** plane tail number: aircraft registration, unique aircraft identifier  
**ActualElapsedTime** in minutes  
**CRSElapsedTime** in minutes  
**AirTime** in minutes  
**ArrDelay** arrival delay, in minutes: A flight is counted as “on time” if it operated less than 15 minutes later the scheduled time shown in the carriers’ Computerized Reservations Systems (CRS).  
**DepDelay** departure delay, in minutes  
**Origin** origin IATA airport code  
**Dest** destination IATA airport code  
**Distance** in miles  
**TaxiIn** taxi in time in minutes  
**TaxiOut** taxi out time in minutes  
**Cancelled** was the flight cancelled, 1 = yes, 0 = no   
**CancellationCode** reason for cancellation (A = carrier, B = weather, C = NAS, D = security)  
**Diverted** 1 = yes, 0 = no  
**CarrierDelay** in minutes: Carrier delay is within the control of the air carrier.  
**WeatherDelay** in minutes: Weather delay is caused by extreme or hazardous weather conditions that are forecasted or manifest themselves on point of departure, enroute, or on point of arrival.  
**NASDelay** in minutes: Delay that is within the control of the National Airspace System (NAS).  
**SecurityDelay** in minutes: Security delay is caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.  
**LateAircraftDelay** in minutes: Arrival delay at an airport due to the late arrival of the same aircraft at a previous airport. The ripple effect of an earlier delay at downstream airports is referred to as delay propagation.  

## **Analize the features to make primary decissions about which Attributes need to be modified and delete those considered redundant or prescindble:**

In [3]:
flights_raw_db.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.00,1955,2211.00,2225,WN,335,N712SW,128.00,150.00,116.00,-14.00,8.00,IAD,TPA,810,4.00,8.00,0,N,0,nan,nan,nan,nan,nan
1,2008,1,3,4,754.00,735,1002.00,1000,WN,3231,N772SW,128.00,145.00,113.00,2.00,19.00,IAD,TPA,810,5.00,10.00,0,N,0,nan,nan,nan,nan,nan
2,2008,1,3,4,628.00,620,804.00,750,WN,448,N428WN,96.00,90.00,76.00,14.00,8.00,IND,BWI,515,3.00,17.00,0,N,0,nan,nan,nan,nan,nan
4,2008,1,3,4,1829.00,1755,1959.00,1925,WN,3920,N464WN,90.00,90.00,77.00,34.00,34.00,IND,BWI,515,3.00,10.00,0,N,0,2.00,0.00,0.00,0.00,32.00
5,2008,1,3,4,1940.00,1915,2121.00,2110,WN,378,N726SW,101.00,115.00,87.00,11.00,25.00,IND,JAX,688,4.00,10.00,0,N,0,nan,nan,nan,nan,nan


In [4]:
flights_raw_db.describe(include="all")

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1936758.00,1936758.00,1936758.00,1936758.00,1936758.00,1936758.00,1929648.00,1936758.00,1936758,1936758.00,1936753,1928371.00,1936560.00,1928371.00,1928371.00,1936758.00,1936758,1936758,1936758.00,1929648.00,1936303.00,1936758.00,1936758,1936758.00,1247488.00,1247488.00,1247488.00,1247488.00,1247488.00
unique,nan,nan,nan,nan,nan,nan,nan,nan,20,nan,5366,nan,nan,nan,nan,nan,303,304,nan,nan,nan,nan,4,nan,nan,nan,nan,nan,nan
top,nan,nan,nan,nan,nan,nan,nan,nan,WN,nan,N325SW,nan,nan,nan,nan,nan,ATL,ORD,nan,nan,nan,nan,N,nan,nan,nan,nan,nan,nan
freq,nan,nan,nan,nan,nan,nan,nan,nan,377602,nan,965,nan,nan,nan,nan,nan,131613,108984,nan,nan,nan,nan,1936125,nan,nan,nan,nan,nan,nan
mean,2008.00,6.11,15.75,3.98,1518.53,1467.47,1610.14,1634.22,NaN,2184.26,NaN,133.31,134.30,108.28,42.20,43.19,NaN,NaN,765.69,6.81,18.23,0.00,NaN,0.00,19.18,3.70,15.02,0.09,25.30
std,0.00,3.48,8.78,2.00,450.49,424.77,548.18,464.63,NaN,1944.70,NaN,72.06,71.34,68.64,56.78,53.40,NaN,NaN,574.48,5.27,14.34,0.02,NaN,0.06,43.55,21.49,33.83,2.02,42.05
min,2008.00,1.00,1.00,1.00,1.00,0.00,1.00,0.00,NaN,1.00,NaN,14.00,-25.00,0.00,-109.00,6.00,NaN,NaN,11.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00
25%,2008.00,3.00,8.00,2.00,1203.00,1135.00,1316.00,1325.00,NaN,610.00,NaN,80.00,82.00,58.00,9.00,12.00,NaN,NaN,338.00,4.00,10.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00
50%,2008.00,6.00,16.00,4.00,1545.00,1510.00,1715.00,1705.00,NaN,1543.00,NaN,116.00,116.00,90.00,24.00,24.00,NaN,NaN,606.00,6.00,14.00,0.00,NaN,0.00,2.00,0.00,2.00,0.00,8.00
75%,2008.00,9.00,23.00,6.00,1900.00,1815.00,2030.00,2014.00,NaN,3422.00,NaN,165.00,165.00,137.00,56.00,53.00,NaN,NaN,998.00,8.00,21.00,0.00,NaN,0.00,21.00,0.00,15.00,0.00,33.00


In [5]:
# Creating a function to show unique values of certain attributes
def unique_values (dataframe, attributes):
   print(f"{[name for name in globals() if globals()[name] is dataframe]}")
   print("----------------")
   for at in attributes: 
      x = dataframe[at].unique()
      print( f"Unique values of {at} are: ", x, f"Counts:{len(x)}")

In [6]:
attributes = ["Year","Cancelled","Diverted","CancellationCode"]
unique_values(flights_raw_db, attributes)

['flights_raw_db']
----------------
Unique values of Year are:  [2008] Counts:1
Unique values of Cancelled are:  [0 1] Counts:2
Unique values of Diverted are:  [0 1] Counts:2
Unique values of CancellationCode are:  ['N' 'A' 'B' 'C'] Counts:4


In [7]:
# Check iF Cancelled flights show valuable information about delays
attributes = ["CancellationCode","DepTime","CRSDepTime","ArrTime","CRSArrTime","ActualElapsedTime","CRSElapsedTime","AirTime","ArrDelay","DepDelay","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]
cancelled = flights_raw_db[attributes][flights_raw_db["Cancelled"] ==  1]
cancelled

,CancellationCode,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
5463024,A,1622.00,1420,nan,1520,nan,60.00,nan,nan,122.00,nan,nan,nan,nan,nan
5484245,B,1323.00,1255,nan,1442,nan,107.00,nan,nan,28.00,nan,nan,nan,nan,nan
5486876,C,1825.00,1815,nan,1927,nan,72.00,nan,nan,10.00,nan,nan,nan,nan,nan
5486924,B,1733.00,1715,nan,1818,nan,63.00,nan,nan,18.00,nan,nan,nan,nan,nan
5491819,B,1943.00,1745,nan,1857,nan,72.00,nan,nan,118.00,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002526,A,1526.00,1444,nan,1654,nan,130.00,nan,nan,42.00,nan,nan,nan,nan,nan
7006018,C,1431.00,1422,nan,1527,nan,125.00,nan,nan,9.00,nan,nan,nan,nan,nan
7006289,A,1459.00,1447,nan,1650,nan,123.00,nan,nan,12.00,nan,nan,nan,nan,nan
7006809,A,1026.00,955,nan,1219,nan,144.00,nan,nan,31.00,nan,nan,nan,nan,nan


In [8]:
attributes = ["ArrDelay","ActualElapsedTime","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]
unique_values(cancelled,attributes)

['_', 'cancelled', '_7']
----------------
Unique values of ArrDelay are:  [nan] Counts:1
Unique values of ActualElapsedTime are:  [nan] Counts:1
Unique values of CarrierDelay are:  [nan] Counts:1
Unique values of WeatherDelay are:  [nan] Counts:1
Unique values of NASDelay are:  [nan] Counts:1
Unique values of SecurityDelay are:  [nan] Counts:1
Unique values of LateAircraftDelay are:  [nan] Counts:1


In [9]:
# Check iF Diverted flights show valuable information about delays
attributes = ["Diverted","DepTime","CRSDepTime","ArrTime","CRSArrTime","ActualElapsedTime","CRSElapsedTime","AirTime","ArrDelay","DepDelay","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]
diverted = flights_raw_db[attributes][flights_raw_db["Diverted"] ==  1]
diverted

,Diverted,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1763,1,922.00,915,nan,1050,nan,95.00,nan,nan,7.00,nan,nan,nan,nan,nan
1911,1,2325.00,1900,nan,2030,nan,90.00,nan,nan,265.00,nan,nan,nan,nan,nan
2651,1,1949.00,1905,nan,1910,nan,65.00,nan,nan,44.00,nan,nan,nan,nan,nan
2726,1,737.00,705,nan,825,nan,80.00,nan,nan,32.00,nan,nan,nan,nan,nan
3672,1,1849.00,1630,nan,1755,nan,85.00,nan,nan,139.00,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7001470,1,1928.00,1645,29.00,2032,nan,407.00,nan,nan,163.00,nan,nan,nan,nan,nan
7004192,1,1957.00,1905,22.00,2013,nan,128.00,nan,nan,52.00,nan,nan,nan,nan,nan
7006200,1,714.00,640,1153.00,859,nan,79.00,nan,nan,34.00,nan,nan,nan,nan,nan
7006401,1,1355.00,1106,7.00,1950,nan,344.00,nan,nan,169.00,nan,nan,nan,nan,nan


In [10]:
attributes = ["ArrDelay","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]
unique_values(diverted, attributes)

['_', 'diverted', '_9']
----------------
Unique values of ArrDelay are:  [nan] Counts:1
Unique values of CarrierDelay are:  [nan] Counts:1
Unique values of WeatherDelay are:  [nan] Counts:1
Unique values of NASDelay are:  [nan] Counts:1
Unique values of SecurityDelay are:  [nan] Counts:1
Unique values of LateAircraftDelay are:  [nan] Counts:1


As expected, Cancelled and Diverted flights doesn't have information about delays as they don't detach from the airport. So, cancelled and delayed flights must be deleted as well as the attributes related to the cancellantion ("Cancelles","Cancellation Code" and Diverted) to avoid noise on our dataframe.  

For Diverted Flights another possible approach is calculating the difference between ArrTime and CRSArrTime " ArrDelay = ArrTime - CRSArrTime". I've discarted this approach as in the original Dataframe this aren't cosidered delayed Flights.

Also, All the flights logged on the DF occur in 2008, "Year" Attribute dosesn't hany valuable information and it must be deleted. 

In [11]:
flights_db = flights_raw_db.copy()
lables = flights_db[(flights_db.Cancelled ==  1) | (flights_db.Diverted ==  1)].index
flights_db.drop(labels=lables, axis=0,inplace=True)
flights_db.drop(axis=1, columns= ["Year", "Cancelled", "CancellationCode", "Diverted"], inplace = True)
flights_db

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,3,4,2003.00,1955,2211.00,2225,WN,335,N712SW,128.00,150.00,116.00,-14.00,8.00,IAD,TPA,810,4.00,8.00,nan,nan,nan,nan,nan
1,1,3,4,754.00,735,1002.00,1000,WN,3231,N772SW,128.00,145.00,113.00,2.00,19.00,IAD,TPA,810,5.00,10.00,nan,nan,nan,nan,nan
2,1,3,4,628.00,620,804.00,750,WN,448,N428WN,96.00,90.00,76.00,14.00,8.00,IND,BWI,515,3.00,17.00,nan,nan,nan,nan,nan
4,1,3,4,1829.00,1755,1959.00,1925,WN,3920,N464WN,90.00,90.00,77.00,34.00,34.00,IND,BWI,515,3.00,10.00,2.00,0.00,0.00,0.00,32.00
5,1,3,4,1940.00,1915,2121.00,2110,WN,378,N726SW,101.00,115.00,87.00,11.00,25.00,IND,JAX,688,4.00,10.00,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1250.00,1220,1617.00,1552,DL,1621,N938DL,147.00,152.00,120.00,25.00,30.00,MSP,ATL,906,9.00,18.00,3.00,0.00,0.00,0.00,22.00
7009717,12,13,6,657.00,600,904.00,749,DL,1631,N3743H,127.00,109.00,78.00,75.00,57.00,RIC,ATL,481,15.00,34.00,0.00,57.00,18.00,0.00,0.00
7009718,12,13,6,1007.00,847,1149.00,1010,DL,1631,N909DA,162.00,143.00,122.00,99.00,80.00,ATL,IAH,689,8.00,32.00,1.00,0.00,19.00,0.00,79.00
7009726,12,13,6,1251.00,1240,1446.00,1437,DL,1639,N646DL,115.00,117.00,89.00,9.00,11.00,IAD,ATL,533,13.00,13.00,nan,nan,nan,nan,nan


## **Handling Null Values**

In [12]:
# Showing Null Values only for the Attributes that have one or more.

def null_values (dataframe):
    nulls = {}
    for column in dataframe.columns:
        null = dataframe[column].isnull().sum()
        if null > 0: 
            nulls[column] = null     
    if len(nulls.keys()) == 0:
        print("There is any Null Value in the Dataframe")
    else:
        for key in nulls: 
            print(f"{key}: {nulls[key]}")

In [13]:
null_values(flights_db)

TailNum: 3
CarrierDelay: 680883
WeatherDelay: 680883
NASDelay: 680883
SecurityDelay: 680883
LateAircraftDelay: 680883


In [14]:
# Replace Null Values for CarrierDelay, WeatherDelay, NASDelay, SecurityDelay and LateAircraftDelay per 0, as this attributes are exppressed in minutes, hence NaN means 0 minuts.
attributes = ["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"]
flights_db[attributes] = flights_db[attributes].fillna(0)
null_values(flights_db)

TailNum: 3


In [15]:
# Extracting the amount of rows in the dataframe to calculate the ratio on null data for the attribute Airtime.
shape = flights_db.shape
shaperatio = flights_db["TailNum"].isnull().sum() /shape[0] * 100
print(f"Rows with null data represents {shaperatio}% of the total")

Rows with null data represents 0.00015557172349096724% of the total


The 3 rows with null values in "TailNum" represent a 0.0001% of the dataset .Best approach would be delete those indexes are they have no practical significante. Anyway In this exercise we are going to use a method to fill the NaN values with an estatistical aproximation.

In [16]:
# Filling the null values from the dataframe
flights_db["TailNum"].fillna(method="pad", inplace=True)
null_values(flights_db)

There is any Null Value in the Dataframe


In [17]:
flights_convert = flights_db.copy()
flights_convert

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,3,4,2003.00,1955,2211.00,2225,WN,335,N712SW,128.00,150.00,116.00,-14.00,8.00,IAD,TPA,810,4.00,8.00,0.00,0.00,0.00,0.00,0.00
1,1,3,4,754.00,735,1002.00,1000,WN,3231,N772SW,128.00,145.00,113.00,2.00,19.00,IAD,TPA,810,5.00,10.00,0.00,0.00,0.00,0.00,0.00
2,1,3,4,628.00,620,804.00,750,WN,448,N428WN,96.00,90.00,76.00,14.00,8.00,IND,BWI,515,3.00,17.00,0.00,0.00,0.00,0.00,0.00
4,1,3,4,1829.00,1755,1959.00,1925,WN,3920,N464WN,90.00,90.00,77.00,34.00,34.00,IND,BWI,515,3.00,10.00,2.00,0.00,0.00,0.00,32.00
5,1,3,4,1940.00,1915,2121.00,2110,WN,378,N726SW,101.00,115.00,87.00,11.00,25.00,IND,JAX,688,4.00,10.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1250.00,1220,1617.00,1552,DL,1621,N938DL,147.00,152.00,120.00,25.00,30.00,MSP,ATL,906,9.00,18.00,3.00,0.00,0.00,0.00,22.00
7009717,12,13,6,657.00,600,904.00,749,DL,1631,N3743H,127.00,109.00,78.00,75.00,57.00,RIC,ATL,481,15.00,34.00,0.00,57.00,18.00,0.00,0.00
7009718,12,13,6,1007.00,847,1149.00,1010,DL,1631,N909DA,162.00,143.00,122.00,99.00,80.00,ATL,IAH,689,8.00,32.00,1.00,0.00,19.00,0.00,79.00
7009726,12,13,6,1251.00,1240,1446.00,1437,DL,1639,N646DL,115.00,117.00,89.00,9.00,11.00,IAD,ATL,533,13.00,13.00,0.00,0.00,0.00,0.00,0.00


## **Transforming Dtypes**

In [18]:
flights_convert.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928371 entries, 0 to 7009727
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Month              int64  
 1   DayofMonth         int64  
 2   DayOfWeek          int64  
 3   DepTime            float64
 4   CRSDepTime         int64  
 5   ArrTime            float64
 6   CRSArrTime         int64  
 7   UniqueCarrier      object 
 8   FlightNum          int64  
 9   TailNum            object 
 10  ActualElapsedTime  float64
 11  CRSElapsedTime     float64
 12  AirTime            float64
 13  ArrDelay           float64
 14  DepDelay           float64
 15  Origin             object 
 16  Dest               object 
 17  Distance           int64  
 18  TaxiIn             float64
 19  TaxiOut            float64
 20  CarrierDelay       float64
 21  WeatherDelay       float64
 22  NASDelay           float64
 23  SecurityDelay      float64
 24  LateAircraftDelay  float64
dtypes: float64(14), in

In [19]:
# Analizing the dataframe can be showned that all the floats have no values on their decimal possition. In this context the Dtype = Float64 will be converted to integers. 
floats = list(flights_convert.select_dtypes(include="Float64").columns)
unique_decimals = (flights_convert[floats] - flights_convert[floats].astype("int64"))
unique_values(unique_decimals,floats)

['unique_decimals']
----------------
Unique values of DepTime are:  [0.] Counts:1
Unique values of ArrTime are:  [0.] Counts:1
Unique values of ActualElapsedTime are:  [0.] Counts:1
Unique values of CRSElapsedTime are:  [0.] Counts:1
Unique values of AirTime are:  [0.] Counts:1
Unique values of ArrDelay are:  [0.] Counts:1
Unique values of DepDelay are:  [0.] Counts:1
Unique values of TaxiIn are:  [0.] Counts:1
Unique values of TaxiOut are:  [0.] Counts:1
Unique values of CarrierDelay are:  [0.] Counts:1
Unique values of WeatherDelay are:  [0.] Counts:1
Unique values of NASDelay are:  [0.] Counts:1
Unique values of SecurityDelay are:  [0.] Counts:1
Unique values of LateAircraftDelay are:  [0.] Counts:1


In [20]:
# Transforming Float values to Integers
flights_convert[floats] = flights_convert[floats].astype("int64")
flights_convert["FlightNum"] = flights_convert["FlightNum"].astype("object")
flights_convert.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928371 entries, 0 to 7009727
Data columns (total 25 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Month              int64 
 1   DayofMonth         int64 
 2   DayOfWeek          int64 
 3   DepTime            int64 
 4   CRSDepTime         int64 
 5   ArrTime            int64 
 6   CRSArrTime         int64 
 7   UniqueCarrier      object
 8   FlightNum          object
 9   TailNum            object
 10  ActualElapsedTime  int64 
 11  CRSElapsedTime     int64 
 12  AirTime            int64 
 13  ArrDelay           int64 
 14  DepDelay           int64 
 15  Origin             object
 16  Dest               object
 17  Distance           int64 
 18  TaxiIn             int64 
 19  TaxiOut            int64 
 20  CarrierDelay       int64 
 21  WeatherDelay       int64 
 22  NASDelay           int64 
 23  SecurityDelay      int64 
 24  LateAircraftDelay  int64 
dtypes: int64(20), object(5)
memory usage: 382.5+ 

In [21]:
# # Converting Departure, Arrival and Elapsed Time to Hours and Minutes.
# attributes = ['DepTime','CRSDepTime','ArrTime','CRSArrTime']
# flights_convert[attributes] = flights_convert[attributes].astype("str")

# for att in attributes:
#     flights_convert[att][flights_convert[att] >= 2400] = "0000" # Convertim els valors marcats com a 2400 a 0000
#     flights_convert[at] = flights_convert[at].apply(lambda x: x.zfill(4)) # afegim leading zeros a les posicions que falten fina tenir tots el numeros a un format "mcdu"
#     flights_convert[at] = pd.to_datetime(flights_convert[at],format="%H%M").dt.time

# flights_convert[attributes]

In [22]:
# Shaving the dataframe to .CSV
flights_ready = flights_convert.copy()
flights_ready.to_csv("../db/Flights/Flights_Ready.csv")

In [23]:
flights_ready

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,0,0,0,0
1,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,0,0,0,0
2,1,3,4,628,620,804,750,WN,448,N428WN,96,90,76,14,8,IND,BWI,515,3,17,0,0,0,0,0
4,1,3,4,1829,1755,1959,1925,WN,3920,N464WN,90,90,77,34,34,IND,BWI,515,3,10,2,0,0,0,32
5,1,3,4,1940,1915,2121,2110,WN,378,N726SW,101,115,87,11,25,IND,JAX,688,4,10,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1250,1220,1617,1552,DL,1621,N938DL,147,152,120,25,30,MSP,ATL,906,9,18,3,0,0,0,22
7009717,12,13,6,657,600,904,749,DL,1631,N3743H,127,109,78,75,57,RIC,ATL,481,15,34,0,57,18,0,0
7009718,12,13,6,1007,847,1149,1010,DL,1631,N909DA,162,143,122,99,80,ATL,IAH,689,8,32,1,0,19,0,79
7009726,12,13,6,1251,1240,1446,1437,DL,1639,N646DL,115,117,89,9,11,IAD,ATL,533,13,13,0,0,0,0,0
